In [ ]:
    def optimize_SLSQP(objective_function, start_weights, bounds, constraints, args=()):

        result = sco.minimize(objective_function, start_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)
        optimized_weights = result.x
        optimized_value = result.fun
       

        return optimized_weights, optimized_value

    def portfolio_autocorr_penalty(self, weights):
        # Calculate portfolio returns based on weights and asset returns
        portfolio_returns = np.dot(self.asset_returns, weights)

        # Calculate the autocorrelation coefficient for the portfolio returns
        num = len(portfolio_returns)
        coef = np.abs(np.corrcoef(portfolio_returns[:-1], portfolio_returns[1:])[0, 1])
        corr = [((num - x) / num) * coef**x for x in range(1, num)]

        # Calculate and return the autocorrelation penalty
        return np.sqrt(1 + 2 * np.sum(corr))

    def neg_sharpe_ratio_ff(self, weights, *args):
        if self.ff_expected_returns is None or self.ff_expected_returns.empty:
            return float('inf')
    
        trading_days = 252
    
        daily_excess_return = np.dot(weights, self.ff_expected_returns) - self.rf_daily
        annual_excess_return = daily_excess_return * trading_days
    
        daily_volatility = self.portfolio_volatility(weights, self.asset_cov_matrix)
        annual_volatility = daily_volatility * np.sqrt(trading_days)
    
        annual_sharpe_ratio_ff = annual_excess_return / annual_volatility
    
        return -annual_sharpe_ratio_ff
    

    def neg_omega_ratio(self, weights, Smart=False):

        portfolio_returns = pd.DataFrame(self.asset_returns.dot(weights))
        excess_returns = portfolio_returns[0] - self.benchmark_returns[self.benchmark_returns.columns[0]] 
        positive_excess = excess_returns[excess_returns > 0].sum()
        negative_excess = -excess_returns[excess_returns < 0].sum()
        omega_ratio = positive_excess / negative_excess
    
       
        if Smart:
            autocorr_penalty = self.portfolio_autocorr_penalty(weights)
            omega_ratio /= (1 + ((autocorr_penalty - 1) * 2))
    
        return -omega_ratio

  def neg_sortino_ratio(self, weights, *args):
        
        portfolio_return = np.dot(weights, self.average_asset_returns)
        excess_returns = self.asset_returns - self.rf_daily
        negative_excess_returns = excess_returns[excess_returns < 0]
        weighted_negative_excess_returns = negative_excess_returns.multiply(weights, axis=1)
        semivariance = np.mean(np.square(weighted_negative_excess_returns.sum(axis=1)))

        sortino_ratio = (portfolio_return - self.rf_daily) / np.sqrt(semivariance)
        
        return -sortino_ratio

    def cvar(self, weights, alpha=0.05, Smart=False):

        # Step 1: Calculate the portfolio returns
        portfolio_returns = self.asset_returns.dot(weights)

        # Step 2: Calculate VaR at the specified alpha quantile
        VaR = np.percentile(portfolio_returns, alpha * 100)
    
        # Step 3: Calculate CVaR as the mean of the returns below the VaR threshold
        CVaR = portfolio_returns[portfolio_returns <= VaR].mean()
    
        return CVaR


























